In [5]:
import math
import pandas as pd

# Modify here
num_readlines = math.inf

# Given Data
obsolete_GOs = ['GO:0046658', 'GO:0005779', 'GO:0015299', 'GO:0031305', 'GO:0030176', 'GO:0097056', 'GO:0099056', 'GO:0006471', 'GO:0008022', 'GO:0031225', 'GO:0140289', 'GO:0005639', 'GO:0046916', 'GO:0099055', 'GO:0030176']
taxon_to_clade = {'cellular organisms': 'cellular organisms', 'Eukaryota': 'Eukaryota', 'Opisthokonta': 'Eukaryota', 'Metazoa': 'Metazoa', 'Eumetazoa': 'Metazoa', 'Bilateria': 'Metazoa', 'Deuterostomia': 'Metazoa', 'Chordata': 'Chordata', 'Craniata': 'Chordata', 'Vertebrata': 'Chordata', 'Gnathostomata': 'Chordata', 'Teleostomi': 'Chordata', 'Euteleostomi': 'Chordata', 'Sarcopterygii': 'Chordata', 'Dipnotetrapodomorpha': 'Chordata', 'Tetrapoda': 'Chordata', 'Amniota': 'Chordata', 'Mammalia': 'Mammalia', 'Theria': 'Mammalia', 'Eutheria': 'Mammalia', 'Boreoeutheria': 'Mammalia', 'Euarchontoglires': 'Mammalia', 'Primates': 'Primates', 'Haplorrhini': 'Primates', 'Simiiformes': 'Primates', 'Catarrhini': 'Primates', 'Hominoidea': 'Primates', 'Hominidae': 'Hominidae', 'Homininae': 'Hominidae', 'Homo': 'Homo', 'Homo sapiens': 'Homo sapiens'}


fn = "human-higher-pLddt-repId_nMem_repLen_avgLen_varLen_repPlddt_avgPlddt_varPlddt_taxId_rank_rankName_link_sapId-sapPlddt-sapGO.tsv"
fn = "clu-sap-repId_lcaTaxId_lcaRankName_sapId_sapPlddt_sapGO.tsv"
f = open(f"../homo_sapiens/{fn}")

# internal
go_diversity = set()
go_frequency = {}

# output data
data = {'repId': [], 'sapId': [], 'lca_clade': [], 'GO_function': [], 'lca_taxon': []}
pd_data = pd.DataFrame(columns = ['organism', 'rank', 'GO_function'])

n = 0
while True:
    line = f.readline().strip()
    n += 1
    
    if n >= num_readlines:
        break
    if not line:
        break
        
    tokens = line.split("\t")

    repId = tokens[0]
    lca_taxon = tokens[2]
    sapId = tokens[3]
    go_function = tokens[5]

    if go_function in obsolete_GOs:
        continue
    
    data['repId'].append(repId)
    data['sapId'].append(sapId)
    data['lca_clade'].append(taxon_to_clade[lca_taxon])
    data['lca_taxon'].append(lca_taxon)
    data['GO_function'].append(go_function)
    
    # generate internal data for analysis
    go_diversity.add(go_function)
    if not go_frequency.get(go_function):
        go_frequency[go_function]=1
    else:
        go_frequency[go_function]+=1

f.close()
pd_data = pd.DataFrame(data)
print(len(go_diversity))
print(len(data['lca_taxon']))

9683
56815


In [6]:
pd_data

,repId,sapId,lca_clade,GO_function,lca_taxon
0,AF-A0A212DI27-F1-model_v3.cif,AF-Q4AC99-F1-model_v3.cif,cellular organisms,GO:0030170,cellular organisms
1,AF-A0A212DI27-F1-model_v3.cif,AF-Q4AC99-F1-model_v3.cif,cellular organisms,GO:0008483,cellular organisms
2,AF-A0A212DI27-F1-model_v3.cif,AF-Q4AC99-F1-model_v3.cif,cellular organisms,GO:0009058,cellular organisms
3,AF-A0A212DI27-F1-model_v3.cif,AF-Q4AC99-F1-model_v3.cif,cellular organisms,GO:0006520,cellular organisms
4,AF-A0A453G0Q8-F1-model_v3.cif,AF-P30153-F1-model_v3.cif,cellular organisms,GO:0000775,cellular organisms
...,...,...,...,...,...
56810,AF-A0A453PE91-F1-model_v3.cif,AF-B4E108-F1-model_v3.cif,Eukaryota,GO:0046872,Eukaryota
56811,AF-A0A453PE91-F1-model_v3.cif,AF-B4E108-F1-model_v3.cif,Eukaryota,GO:0006002,Eukaryota
56812,AF-B7Z6L1-F1-model_v3.cif,AF-B7Z6L1-F1-model_v3.cif,Chordata,GO:0016459,Euteleostomi
56813,AF-B7Z6L1-F1-model_v3.cif,AF-B7Z6L1-F1-model_v3.cif,Chordata,GO:0003779,Euteleostomi


In [7]:
from goatools.base import get_godag
from goatools.gosubdag.gosubdag import GoSubDag
from goatools.godag.go_tasks import get_go2parents, get_go2parents_isa
from goatools.obo_parser import GODag

godag = get_godag('go-basic.obo', optional_attrs="relationship")
output = {'repId': [], 'sapId': [], 'GO_function': [], 'lca_taxon': []}

n = 0
cycle = len(pd_data['GO_function'])
# cycle = 100

for i in range(cycle):
    if i / cycle * 100 > n:
        print(f"percentage: {i/cycle*100}%")
        n+=1
    
    goid = pd_data['GO_function'][i]
    rank = pd_data['lca_clade'][i]
    repId = pd_data['repId'][i]
    sapId = pd_data['sapId'][i]
    lca = pd_data['lca_taxon'][i]

    gosubdag = GoSubDag(goid, godag, relationships=True, prt=False)

    try:
      ntgo = gosubdag.go2nt[goid]
    except:
      print(f"error {goid}")
      continue

    go2parents = get_go2parents_isa(gosubdag.go2obj)
    
    s = [goid]
    traversed = {}
    
    while s:
        cur = s.pop()
        
        if cur == 'GO:0002376':
            output['repId'].append(repId)
            output['sapId'].append(sapId)
            output['GO_function'].append(goid)
            output['lca_taxon'].append(lca)
            break
        try:
          cur_function = godag[cur].name
        except:
           print(f"error in the tree of {goid} with {cur}")
           continue
        
        if traversed.get(cur):
            continue
        traversed[cur] = rank
        
        
        if not go2parents.get(cur):
            continue
            
        for next in go2parents[cur]:
            s.append(next)

print(f"percentage: 100%")

  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2023-04-01) 46,575 Terms; optional_attrs(relationship)
percentage: 0.001760098565519669%
1 GO IDs NOT FOUND IN GO DAG: GO:0030173
error GO:0030173
1 GO IDs NOT FOUND IN GO DAG: GO:0031307
error GO:0031307
1 GO IDs NOT FOUND IN GO DAG: GO:0033561
error GO:0033561
percentage: 1.0014960837806917%
1 GO IDs NOT FOUND IN GO DAG: GO:2001161
error GO:2001161
percentage: 2.001232068995864%
1 GO IDs NOT FOUND IN GO DAG: GO:0044214
error GO:0044214
1 GO IDs NOT FOUND IN GO DAG: GO:0100002
error GO:0100002
1 GO IDs NOT FOUND IN GO DAG: GO:0007568
error GO:0007568
percentage: 3.0009680542110355%
1 GO IDs NOT FOUND IN GO DAG: GO:0007568
error GO:0007568
1 GO IDs NOT FOUND IN GO DAG: GO:0000737
error GO:0000737
percentage: 4.000704039426208%
percentage: 5.00044002464138%
1 GO IDs NOT FOUND IN GO DAG: GO:0031227
error GO:0031227
percentage: 6.000176009856552%
1 GO IDs NOT FOUND IN GO DAG: GO:0071556
error GO:0071556
1 GO IDs NOT FOUND IN GO DAG: GO:200

1 GO IDs NOT FOUND IN GO DAG: GO:0030173
error GO:0030173
1 GO IDs NOT FOUND IN GO DAG: GO:0030173
error GO:0030173
1 GO IDs NOT FOUND IN GO DAG: GO:0047485
error GO:0047485
percentage: 52.000352019713105%
1 GO IDs NOT FOUND IN GO DAG: GO:0035574
error GO:0035574
percentage: 53.00008800492828%
1 GO IDs NOT FOUND IN GO DAG: GO:0046939
error GO:0046939
percentage: 54.00158408870897%
1 GO IDs NOT FOUND IN GO DAG: GO:0007568
error GO:0007568
1 GO IDs NOT FOUND IN GO DAG: GO:0018024
error GO:0018024
1 GO IDs NOT FOUND IN GO DAG: GO:2001255
error GO:2001255
1 GO IDs NOT FOUND IN GO DAG: GO:1905437
error GO:1905437
percentage: 55.00132007392414%
1 GO IDs NOT FOUND IN GO DAG: GO:0031362
error GO:0031362
1 GO IDs NOT FOUND IN GO DAG: GO:1901675
error GO:1901675
1 GO IDs NOT FOUND IN GO DAG: GO:1901675
error GO:1901675
percentage: 56.00105605913931%
1 GO IDs NOT FOUND IN GO DAG: GO:0098993
error GO:0098993
percentage: 57.00079204435448%
1 GO IDs NOT FOUND IN GO DAG: GO:0030173
error GO:0030173
1

In [10]:
print(output)

{'repId': ['AF-A0A5K4FB41-F1-model_v3.cif', 'AF-A0A6G0TTL2-F1-model_v3.cif', 'AF-A0A498LQC5-F1-model_v3.cif', 'AF-A0A498LQC5-F1-model_v3.cif', 'AF-A0A6P8G6G5-F1-model_v3.cif', 'AF-A0A4X2JZY5-F1-model_v3.cif', 'AF-A0A286S9Y4-F1-model_v3.cif', 'AF-A0A286S9Y4-F1-model_v3.cif', 'AF-A0A286S9Y4-F1-model_v3.cif', 'AF-A0A286S9Y4-F1-model_v3.cif', 'AF-T1JFE5-F1-model_v3.cif', 'AF-D3ZV49-F1-model_v3.cif', 'AF-D3ZV49-F1-model_v3.cif', 'AF-D3ZV49-F1-model_v3.cif', 'AF-D3ZV49-F1-model_v3.cif', 'AF-D3ZV49-F1-model_v3.cif', 'AF-A0A4Y9FT94-F1-model_v3.cif', 'AF-A0A6P4T4H1-F1-model_v3.cif', 'AF-A0A6P4T4H1-F1-model_v3.cif', 'AF-A0A6A2WU80-F1-model_v3.cif', 'AF-A0A6A2WU80-F1-model_v3.cif', 'AF-A0A507F0Z4-F1-model_v3.cif', 'AF-A0A7C4C840-F1-model_v3.cif', 'AF-A0A7C4C840-F1-model_v3.cif', 'AF-A0A2K3CX85-F1-model_v3.cif', 'AF-Q55DL8-F1-model_v3.cif', 'AF-A0A4X2L807-F1-model_v3.cif', 'AF-A0A3M7RS47-F1-model_v3.cif', 'AF-A0A3M7RS47-F1-model_v3.cif', 'AF-A0A2R7WKF9-F1-model_v3.cif', 'AF-A0A1I8F4V4-F1-model_v3.

In [9]:
fo = open('../homo_sapiens/human-immune-go-repId_sapId_sapGO_lca_sapGOFunc.tsv', 'w')

for i in range(len(output['lca_taxon'])):
    goid = output['GO_function'][i]
    fo.write(f"{output['repId'][i]}\t{output['sapId'][i]}\t{output['GO_function'][i]}\t{output['lca_taxon'][i]}\t{godag[goid].name}\n")
fo.close()